In [96]:
import scipy.io
import numpy as np
import CMR_IA as cmr
import matplotlib.pyplot as plt
import pickle
from statistics import mean, stdev
import pprint
# import import_ipynb
import time
import pandas as pd
import seaborn as sns

np.set_printoptions(edgeitems=20, linewidth=150,precision=4)

In [97]:
with open("../Data/simu5_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)

In [98]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [99]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.5,
    beta_rec = 0.5,
    beta_rec_post = 0.1,
    beta_distract = 0.1,
    gamma_fc = 0.5,
    gamma_cf = 0.5,
    s_fc = 0.1,
    s_cf = 0.1,
    c_thresh = 0.01,
    c_thresh_ass = 0.01,
    a = 2800,
    b = 2.14,
    m = 0,
    n = 1,
    kappa = 0.02,
    lamb = 0.01,
    eta = 0,
    phi_s = 10,
    phi_d = 0.5,
    nitems_in_accumulator = 10,
    d_ass = 1,
    learn_while_retrieving = False
)
params

{'beta_enc': 0.5,
 'beta_rec': 0.5,
 'beta_cue': 0.5,
 'beta_rec_new': 0.5,
 'beta_rec_post': 0.1,
 'beta_distract': 0.1,
 'phi_s': 10,
 'phi_d': 0.5,
 's_cf': 0.1,
 's_fc': 0.1,
 'kappa': 0.02,
 'eta': 0,
 'omega': 8,
 'alpha': 1,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.01,
 'd_ass': 1,
 'lamb': 0.01,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 10,
 'max_recalls': 50,
 'learn_while_retrieving': False,
 'a': 2800,
 'b': 2.14,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.5,
 'gamma_cf': 0.5}

In [100]:
df_study = df_study.loc[df_study.session == 0]
# df_study['study_itemno1'] = [1,3,5,7,9,11,13,15,17,19,21,23]
# df_study['study_itemno2'] = [2,4,6,8,10,12,14,16,18,20,22,24]
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,list,session
0,716,141,HOOP,BLUEJAY,1,0
1,1261,862,SEAGULL,MAN,1,0
2,1067,1402,PIKE,SUIT,1,0
3,146,707,BOLT,HIGHWAY,1,0
4,155,1140,BOULDER,PUDDLE,1,0
5,1599,330,WEED,CLOWN,1,0


In [101]:
df_test = df_test.loc[df_test.session == 0]
# df_test['test_itemno'] = [1]
# df_test['correct_ans'] = [2]
df_test

,test_itemno,test_item,correct_ans,lag,list,session
0,716,HOOP,141,5,1,0


In [102]:
df_thin = df_test[['session','test_itemno']]
df_thin = df_thin.assign(s_resp=np.nan, s_rt=np.nan)
df_thin

,session,test_itemno,s_resp,s_rt
0,0,716,NaN,NaN


In [103]:
# extarct the session data
sess = 0
pres_mat = df_study.loc[df_study.session==sess,['study_itemno1','study_itemno2']].to_numpy()
pres_mat = np.reshape(pres_mat,(1,len(pres_mat),2))
pres_mat

array([[[ 716,  141],
        [1261,  862],
        [1067, 1402],
        [ 146,  707],
        [ 155, 1140],
        [1599,  330]]])

In [104]:
cue_mat = df_thin.loc[df_thin.session==sess,'test_itemno'].to_numpy()
cue_mat = np.reshape(cue_mat,(1,-1))
cue_mat

array([[716]])

In [105]:
sem_mat = s_mat

In [106]:
model = cmr.CMR2(params, pres_mat, sem_mat, source_mat=None,
           rec_mat=None, ffr_mat=None, cue_mat=cue_mat, task='CR', mode='Final')

In [107]:
model.pres_nos

array([[[ 716,  141],
        [1261,  862],
        [1067, 1402],
        [ 146,  707],
        [ 155, 1140],
        [1599,  330]]], dtype=int16)

In [108]:
model.cues_nos

array([[716]], dtype=int16)

In [109]:
model.all_nos_unique

array([ 141,  146,  155,  330,  707,  716,  862, 1067, 1140, 1261, 1402, 1599], dtype=int16)

In [110]:
model.pres_indexes

array([[[ 5,  0],
        [ 9,  6],
        [ 7, 10],
        [ 1,  4],
        [ 2,  8],
        [11,  3]]], dtype=int64)

In [111]:
model.cues_indexes

array([[5]], dtype=int64)

In [112]:
model.nlists

1

In [113]:
model.ndistractors

3

In [114]:
model.cues_indexes.shape[1]

1

In [115]:
model.run_norm_cr_single_sess()

# in model:
# print(self.att_vec[self.all_nos_unique[item_idx]-1])
# print(np.dot(self.c_old[:self.nitems_unique], self.f[:self.nitems_unique].T))

[11.      7.0653  4.6788  3.2313  2.3534  1.8208]
11.0
[11.      7.0653  4.6788  3.2313  2.3534  1.8208]
7.065306597126334
[11.      7.0653  4.6788  3.2313  2.3534  1.8208]
4.678794411714423
[11.      7.0653  4.6788  3.2313  2.3534  1.8208]
3.231301601484298
[11.      7.0653  4.6788  3.2313  2.3534  1.8208]
2.3533528323661272
[11.      7.0653  4.6788  3.2313  2.3534  1.8208]
1.820849986238988


In [116]:
len(model.att_vec)

1638

In [117]:
model.ret_thresh

array([9., 1., 1., 1., 1., 9., 1., 1., 1., 1., 1., 1.], dtype=float32)

In [118]:
model.rec_items

[141]

In [119]:
model.rec_times

[350]

In [120]:
model.M_FC

array([[0.5   , 0.1492, 0.1349, 0.1214, 0.1469, 0.5153, 0.1858, 0.1704, 0.1374, 0.1994, 0.1642, 0.1203, 0.    , 0.    , 0.    ],
       [0.0088, 0.5147, 0.201 , 0.1634, 0.5203, 0.0135, 0.0125, 0.0203, 0.1948, 0.0167, 0.0169, 0.1619, 0.    , 0.    , 0.    ],
       [0.0125, 0.0286, 0.5219, 0.1993, 0.0278, 0.0142, 0.0204, 0.0232, 0.54  , 0.0183, 0.0174, 0.2019, 0.    , 0.    , 0.    ],
       [0.0132, 0.0236, 0.024 , 0.5201, 0.0222, 0.0092, 0.0229, 0.0201, 0.0283, 0.02  , 0.0272, 0.526 , 0.    , 0.    , 0.    ],
       [0.0065, 0.5126, 0.1937, 0.1571, 0.5071, 0.0052, 0.0097, 0.012 , 0.1903, 0.0068, 0.0055, 0.1566, 0.    , 0.    , 0.    ],
       [0.5153, 0.1449, 0.1303, 0.1112, 0.1366, 0.5   , 0.1799, 0.1578, 0.1262, 0.1846, 0.1542, 0.1063, 0.    , 0.    , 0.    ],
       [0.0098, 0.1603, 0.1445, 0.1299, 0.1611, 0.0038, 0.5046, 0.1852, 0.1427, 0.5162, 0.184 , 0.1204, 0.    , 0.    , 0.    ],
       [0.0106, 0.1942, 0.1679, 0.1381, 0.1908, 0.0051, 0.0105, 0.5101, 0.1659, 0.0165, 0.51  , 0

In [121]:
model.M_CF

array([[5.0000e-01, 8.8198e-03, 1.2539e-02, 1.3209e-02, 6.5019e-03, 5.5153e+00, 9.7579e-03, 1.0576e-02, 1.5044e-02, 2.3299e-02, 4.3100e-03,
        1.2048e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00],
       [4.6238e-01, 5.4751e-01, 6.5203e-02, 6.5857e-02, 1.6440e+00, 4.3814e-01, 5.0706e-01, 6.0721e-01, 6.1154e-02, 5.2900e-01, 5.8545e-01,
        3.7348e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00],
       [3.0051e-01, 4.5634e-01, 5.5155e-01, 4.9102e-02, 4.4044e-01, 2.8739e-01, 3.2482e-01, 3.8377e-01, 1.2397e+00, 3.3461e-01, 3.5872e-01,
        3.3709e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.1025e-01, 2.9360e-01, 3.5836e-01, 5.3666e-01, 2.8341e-01, 1.9500e-01, 2.2399e-01, 2.4797e-01, 3.5857e-01, 2.3091e-01, 2.3783e-01,
        9.4231e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00],
       [4.6006e-01, 1.6687e+00, 6.4313e-02, 6.4436e-02, 5.2278e-01, 4.2979e-01, 5.0791e-01, 6.0384e-01, 6.3043e-02, 5.2277e-01, 5.7895e-01,
        3.6817e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00],
       [5.5153e

In [122]:
recs = model.rec_items
rts = model.rec_times
result = np.column_stack((recs,rts))
df_thin.loc[df_thin.session==sess, ['s_resp','s_rt']] = result
df_simu = df_thin.merge(df_test,on=['session','test_itemno'])
df_simu

,session,test_itemno,s_resp,s_rt,test_item,correct_ans,lag,list
0,0,716,141.0,350.0,HOOP,141,5,1


In [123]:
tmp1 = df_study.study_itemno1.to_numpy()
tmp2 = df_study.study_itemno2.to_numpy()
tmp = np.concatenate((tmp1, tmp2))
tmp = np.sort(tmp)
tmp

array([ 141,  146,  155,  330,  707,  716,  862, 1067, 1140, 1261, 1402, 1599])

In [124]:
testid = np.searchsorted(tmp, df_simu.test_itemno.item())
corrid = np.searchsorted(tmp, df_simu.correct_ans.item())

In [125]:
df_simu['corr_fin']= model.f_in[corrid]
df_simu['othe_fin'] = np.mean(np.delete(model.f_in, [testid,corrid]))
df_simu

AttributeError: 'CMR2' object has no attribute 'f_in'